In [21]:
import sys
import os
import numpy as np
from encode import Chromagram
from encode import Encoder
import pickle

import risp
import neuro
import eons
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder


In [2]:
data_dir = "/home/dofo/Repos/Github/neuromorphicsnn/chord_snn/dataset/chords"

all_files = [os.path.join(root, fname)
             for root, dirs, files in os.walk(data_dir)
             for fname in files if fname.endswith('.wav')]

In [5]:
data_dir = "/home/dofo/Repos/Github/neuromorphicsnn/chord_snn/dataset/chords"

all_files = [os.path.join(root, fname)
             for root, dirs, files in os.walk(data_dir)
             for fname in files if fname.endswith('.wav')]

def parse_filename(filename):
    base_name = os.path.basename(filename)
    parts = base_name.split('_')
    chord_type = parts[1] 
    chord_name = parts[2]
    root_note_name = parts[3]
    return chord_type, chord_name, root_note_name

labels = [parse_filename(fname) for fname in all_files]

In [9]:
spike_data_dir = "/home/dofo/Repos/Github/neuromorphicsnn/chord_snn/dataset/npy_chroma_chords"


for wav_path in all_files:
    chroma = Chromagram(wav_path)
    chroma_array = chroma.chromagram
    
    # Replicate the directory structure for spike data
    relative_path = os.path.relpath(wav_path, data_dir)
    spike_filename = os.path.join(spike_data_dir, relative_path.replace('.wav', '.npy'))
    
    # Ensure the directory exists
    os.makedirs(os.path.dirname(spike_filename), exist_ok=True)
    
    # Save the spike data to disk using pickle
    np.save(spike_filename, chroma_array)

    

/home/dofo/Repos/Bitbucket/framework/pyframework/lib/python3.10/site-packages/librosa/core/spectrum.py:257: UserWarning: n_fft=1024 is too large for input signal of length=691
  warnings.warn(


In [3]:
data_dir = "/home/dofo/Repos/Github/neuromorphicsnn/chord_snn/dataset/npy_chroma_chords"

chroma_files = [os.path.join(root, fname)
             for root, dirs, files in os.walk(data_dir)
             for fname in files if fname.endswith('.npy')]

In [6]:
def create_numerical_labels_for_root(labels):
    # Extract root notes
    root_notes = [root_note for _, root_note, _ in labels]
    
    # Get unique root notes and assign a unique number to each
    unique_root_notes = sorted(list(set(root_notes)))
    root_to_num = {root: i for i, root in enumerate(unique_root_notes)}
    
    # Convert root notes to numerical labels
    numerical_labels = [root_to_num[root] for root in root_notes]
    
    return numerical_labels, root_to_num

# Create numerical labels for root notes
y, root_to_num = create_numerical_labels_for_root(labels)

In [7]:
# First, split the data into training+validation and testing sets
X_temp, X_test, y_temp, y_test = train_test_split(chroma_files, y, test_size=0.2, random_state=42)

# Now, split the training+validation set into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.25, random_state=42)  # 0.25 x 0.8 = 0.2

# One-hot encode the labels
OneHot = OneHotEncoder(sparse_output=False)
y_train_encoded = OneHot.fit_transform(np.array(y_train).reshape(-1, 1))
y_val_encoded = OneHot.transform(np.array(y_val).reshape(-1, 1))
y_test_encoded = OneHot.transform(np.array(y_test).reshape(-1, 1))

In [12]:
y_test_encoded.shape

(968, 88)

In [31]:
class Encoder:
    
    def __init__(self, npy_file, Fs=44100, N=2048, H=1024,):

        self.Fs = Fs                                            # sampling frequency
        self.N = N                                              # FFT size
        self.H = H                                              # hop size

        self.chromagram = np.load(npy_file)                       # chromagram object for us to call on
        
        self.num_frames, self.num_bins, self.time_steps = self.get_num_frames() # get number of frames, bins and time steps
        
        self.chroma_as_list = self._get_chroma()                # chromagram as list
        
        self.encoder_params = {
            "dmin": self.num_bins * [0],                        # a list with the length of the number of bins
            "dmax": self.num_bins * [1],
            "encoders": [{ "rate": { "subinterval_size": self.time_steps } }], #if you change stuff, its this line that you're most likely looking for :)
            "interval": self.time_steps * self.num_frames,      # interval is the length of the song in ms
            }
        
        self.spikes, self.encoder = self._encode()                            # encoded chromagram
        
     
    
    def get_num_frames(self):


        num_frames = self.chromagram.shape[1]
        num_bins = self.chromagram.shape[0]
        time_steps = round( (self.H / self.Fs) * 1000, 1)
        return num_frames, num_bins, time_steps
    
    def _get_chroma(self):
        chroma = self.chromagram             # chromagram as numpy array
        return chroma.tolist()

    def _encode(self):
        encoder = neuro.EncoderArray(self.encoder_params)       # encoder object. "neuro.EncoderArray" from framework
        spikes = encoder.get_timeseries_spikes(self.chroma_as_list) # encoded chromagram. "get_timeseries_spikes" from framework
        return spikes, encoder 

In [11]:
risp_config = {
  "leak_mode": "all",
  "min_weight": -1,
  "max_weight": 1,
  "min_threshold": -1,
  "max_threshold": 1,
  "max_delay": 5,
  "discrete": False
}

proc = risp.Processor(risp_config)

net = neuro.Network()
net.set_properties(proc.get_network_properties())

In [18]:
n_inputs = 12
n_hidden = 100
n_outputs = 88
n_total = n_inputs + n_hidden + n_outputs

moa = neuro.MOA()
moa.seed(42)

In [34]:
for i in range(n_inputs):
    node = net.add_node(i)
    net.add_input(i)
    net.randomize_node_properties(moa,node)

for i in range(n_outputs):
    node = net.add_node(n_inputs+i)
    net.add_output(i)
    net.randomize_node_properties(moa,node)

RuntimeError: Node 0 already exists at specified index.

In [23]:
eo_params = {
    "population_size": 100,
    "starting_nodes": n_total,
    "starting_edges": n_total,
    "multi_edges": 0.05,
    "merge_rate": .01,
    "crossover_rate": 0.5,
    "mutation_rate": 0.9,
    "add_node_rate": 0.5,
    "add_edge_rate": 0.9,
    "delete_node_rate": 0.4,
    "delete_edge_rate": 0.8,
    "node_params_rate": 1.5,
    "edge_params_rate": 1.5,
    "net_params_rate": .01,
    "num_mutations": 5,
    "random_factor": 0.05,
    "num_best": 3,

    "selection_type": "tournament",
    "tournament_size_factor": 0.1,
    "tournament_best_net_factor": 0.9,
    "node_mutations": { "Threshold": 1.0 },
    "net_mutations": { },
    "edge_mutations": { "Weight": 0.5, "Delay": 0.5 },
}


In [24]:
evolver = eons.EONS(eo_params)

In [32]:
ex1_spike = Encoder(ex1).spikes

In [30]:
ex1 ='/home/dofo/Repos/Github/neuromorphicsnn/chord_snn/dataset/npy_chroma_chords/dom7_ninth/NordGrand1_dom7ninth_G3_55.npy'


In [37]:
proc.load_network(net)

True

In [38]:
proc.apply_spikes(Encoder(ex1).spikes)